In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from uuid import uuid4
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
# PDF 로딩
path = "data//raw//청년 법령집.pdf"
loader = PyPDFLoader(file_path=path)
docs = loader.load()
print(type(docs))


for i in range(len(docs)):
    docs[i].page_content = (docs[i].page_content
                            .replace('발  간  등  록  번  호','')
                            .replace('법제처 국가법령정보센터', '')
                            .replace('법제처', '')
                            .replace('Korea Law Service Center', '')
                            .replace('국가법령정보센터', '')
                            .replace('\n', ' ')
                            .replace('\t', ' ')
                            )


<class 'list'>


In [3]:
# Splitter
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o-mini", 
    chunk_size=500,
    chunk_overlap=100
)
docs = splitter.split_documents(docs)

ids = [str(uuid4()) for _ in range(len(docs))]

# Vector Store
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
COLLECTION_NAME = "youth_policy"
PERSIST_DIRECTORY = "vector_store/chroma/youth_policy" #저장할 경로

vector_store = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    ids=ids,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)


In [ ]:
# vector store 연결
vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

In [4]:
# retriever
retriever = vector_store.as_retriever(
        search_type="mmr",
        search_kwargs={"k":10, "fetch_k":10}
)


# Prompt Template 생성
messages = [
        ("ai", """
        당신은 청년 정책 전문가입니다. 아래 문서의 내용으로만 답변해주세요.
        답변을 모르면 모른다고 대답해주세요.
        {context}"""),
        ("human", "{question}"),
]

prompt_template = ChatPromptTemplate(messages)


# 모델
model = ChatOpenAI(model="gpt-4o-mini")


# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt생성) -> model(정답) -> output parser
chain = {"context":retriever, "question":RunnablePassthrough()} | prompt_template | model | parser


In [6]:
while True:
    query = input()
    if query == '!q':
        break
    print('질문:', query) 
    result = chain.invoke(query)
    print('답변', result)
    print('='*50)


질문: 서울의 지원정책을 알려줘
답변 서울특별시는 청년을 위한 다양한 지원 정책을 운영하고 있습니다. 주요 내용은 다음과 같습니다:

1. **청년주거 기본 조례**: 청년의 주거수준 향상과 자립기반 강화를 위한 정책을 수립합니다. 청년의 소득수준에 맞춰 주택공급 및 주거비 지원을 통해 청년이 안정된 주거 환경에서 생활할 수 있도록 합니다.

2. **청년일자리 기본 조례**: 청년의 고용 촉진 및 일자리 질 향상을 위한 정책을 수립합니다. 청년 구직자 실태조사 및 직업 능력 개발 훈련 등을 통해 청년의 취업 지원을 강화합니다.

3. **청년 문화예술 지원**: 청년 문화예술인의 권리 보호 및 육성을 위한 정책을 수립하고, 관련 기관과 협력하여 실태조사 및 자료 요청을 통해 정책의 기초자료를 마련합니다.

이 외에도 서울시는 청년 지원을 위한 다양한 프로그램과 시책을 추진하고 있으며, 이를 통해 청년들이 지역사회에 적극 참여하고 안정적인 삶을 영위할 수 있도록 돕고 있습니다.
질문: 청년 기본법을 알려줘
답변 청년 기본법은 청년의 권리 및 책임과 국가 및 지방자치단체의 청년에 대한 책무를 규정하며, 청년정책의 수립, 조정 및 청년 지원에 관한 기본적인 사항을 담고 있습니다. 주요 내용은 다음과 같습니다:

1. **목적**: 청년의 권리와 책임을 정립하고, 청년정책의 기본 방향을 제시합니다.
2. **기본이념**: 청년이 존엄과 가치를 실현하며 행복한 삶을 영위할 수 있도록 하고, 청년이 적극적으로 사회에 참여할 수 있는 환경을 조성하는 것을 목표로 합니다.
3. **청년 정의**: 청년은 19세 이상 34세 이하의 사람으로 정의되며, 다른 법령에서 다르게 적용될 수 있습니다.
4. **청년정책 기본계획**: 국무총리는 5년마다 청년정책에 관한 기본계획을 수립하고 시행해야 합니다.

이 법은 청년의 정치, 경제, 사회, 문화 등 모든 분야에서의 참여를 촉진하고, 평등한 기회를 제공하는 것을 중요시합니다.
